# 자료형 선언과정

In [1]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U keras-efficientnet-v2
!pip install -U git+https://github.com/leondgarse/keras_efficientnet_v2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leondgarse/keras_efficientnet_v2 to /tmp/pip-req-build-3nbsgf07
  Running command git clone -q https://github.com/leondgarse/keras_efficientnet_v2 /tmp/pip-req-build-3nbsgf07


In [3]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import pickle
import platform
from tensorflow.keras.applications import EfficientNetB0, efficientnet_v2
import keras_efficientnet_v2
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
import matplotlib.pyplot as plt
from keras import backend
from keras import layers
from keras.applications import imagenet_utils
from keras.engine import training
from keras.utils import data_utils
from keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.utils import get_custom_objects
import tensorflow.keras.backend as K
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Input, Dense, Flatten, Activation

In [4]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [5]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
#활성화 함수 mish 
class Mish(Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'
def mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': Mish(mish)})

#데이터 확인

In [6]:
#데이터불러오기
with open('/content/drive/MyDrive/picklefile/trainx.pickle','rb') as f:
    x_train= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/trainy.pickle','rb') as f:
    y_train= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/testx.pickle','rb') as f:
    x_test= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/testy.pickle','rb') as f:
    y_test= pickle.load(f)


In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8635, 224, 224, 3), (8635,), (962, 224, 224, 3), (962,))

In [ ]:
# one hot 
y_train1 = tf.keras.utils.to_categorical(y_train, 3)
y_test1 = tf.keras.utils.to_categorical(y_test, 3)
y_train.shape, y_test.shape

((8635,), (962,))

#전체 재학습


In [9]:
def create_model():
  base_model = efficientnet_v2.EfficientNetV2S(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')
  base_model.trainable = True
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = base_model(inputs, training=False)
  x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  #x= tf.keras.layers.Dropout(0.3)(x)
  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  return tf.keras.Model(inputs, outputs)

In [ ]:
with strategy.scope():
  V2M = create_model()
  V2M.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), 
              loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
  V2M.trainable
  V2M.fit(x_train, y_train, epochs = 80, validation_data=(x_test, y_test), 
                   batch_size= 64)

82420632/82420632 [==============================] - 0s 0us/step
Epoch 1/80
135/135 [==============================] - 133s 293ms/step - loss: 1.1251 - accuracy: 0.3858 - val_loss: 1.0795 - val_accuracy: 0.3825
Epoch 2/80
135/135 [==============================] - 13s 97ms/step - loss: 1.0673 - accuracy: 0.4060 - val_loss: 1.0665 - val_accuracy: 0.4137
Epoch 3/80
135/135 [==============================] - 13s 99ms/step - loss: 1.0397 - accuracy: 0.4366 - val_loss: 1.0650 - val_accuracy: 0.4220
Epoch 4/80
135/135 [==============================] - 13s 98ms/step - loss: 1.0203 - accuracy: 0.4485 - val_loss: 1.0413 - val_accuracy: 0.4241
Epoch 5/80
135/135 [==============================] - 13s 97ms/step - loss: 0.9716 - accuracy: 0.5025 - val_loss: 1.0417 - val_accuracy: 0.4699
Epoch 6/80
135/135 [==============================] - 13s 98ms/step - loss: 0.9113 - accuracy: 0.5430 - val_loss: 1.0157 - val_accuracy: 0.4688
Epoch 7/80
135/135 [==============================] - 13s 99ms/step -

In [ ]:
V2L.summary()

#하위층 일부학습

In [ ]:
#하위층 일부학습
def create_model():
  base_model2 = efficientnet_v2.EfficientNetV2S(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')
  base_model2.trainable = False
  for layer in base_model2.layers[:-150]: 
    layer.trainable = False		
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = base_model2(inputs, training=False)
  x = tf.keras.layers.Flatten(input_shape=base_model2.output_shape[1:])(x)
  x = tf.keras.layers.Dense(128, activation='mish')(x)
  x= tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
  return tf.keras.Model(inputs, outputs)
  

In [ ]:
with strategy.scope():
  model2 = create_model()
  model2.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

  model2.fit(x_train, y_train, epochs = 80, validation_data=(x_test, y_test), 
                   batch_size= 64)

Epoch 1/100
68/68 [==============================] - 160s 1s/step - loss: 1.2643 - accuracy: 0.3529 - val_loss: 1.0781 - val_accuracy: 0.3888
Epoch 2/100
68/68 [==============================] - 7s 109ms/step - loss: 1.0773 - accuracy: 0.3950 - val_loss: 1.0773 - val_accuracy: 0.3877
Epoch 3/100
68/68 [==============================] - 7s 110ms/step - loss: 1.0643 - accuracy: 0.4174 - val_loss: 1.0673 - val_accuracy: 0.4168
Epoch 4/100
68/68 [==============================] - 7s 109ms/step - loss: 1.0506 - accuracy: 0.4313 - val_loss: 1.0740 - val_accuracy: 0.4096
Epoch 5/100
68/68 [==============================] - 7s 109ms/step - loss: 1.0485 - accuracy: 0.4415 - val_loss: 1.0629 - val_accuracy: 0.4106
Epoch 6/100
68/68 [==============================] - 7s 108ms/step - loss: 1.0384 - accuracy: 0.4461 - val_loss: 1.0661 - val_accuracy: 0.4179
Epoch 7/100
68/68 [==============================] - 8s 111ms/step - loss: 1.0257 - accuracy: 0.4574 - val_loss: 1.0623 - val_accuracy: 0.4262


#상위층 일부학습


In [ ]:
#상위층 일부학습
def create_model_1():
  base_model = efficientnet_v2.EfficientNetV2S(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')
  base_model.trainable = False
  inputs = tf.keras.Input(shape=(224, 224, 3))

  x = base_model(inputs, training=False) #traing false 는 trainable과는 다른의미다.
  x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x) #bottlenet feature 3차원 -> 1차원으로 바꾸어야한다
  x = tf.keras.layers.Dense(128, activation='mish')(x) #(1,1,2048)인데 매트릭스 차원이 3차원이기 때문에 1차원으로 수정 Flatten 해주어야한다
  #x= tf.keras.layers.Dropout())(x)                    #이전층이 다음층과 연결이 된다 x->(x) 요느낌 잘 기억! #outputshape(x,y,채널)
  outputs = tf.keras.layers.Dense(3, activation='softmax')(x) #라벨이 3개면  dense 3개

  return tf.keras.Model(inputs, outputs)

In [ ]:
y_train1[1]

array([0., 0., 1.], dtype=float32)

In [ ]:
# categorical_crossentropy
with strategy.scope():
  model = create_model_1()
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), 
                    loss = 'categorical_crossentropy', 
                    metrics=['accuracy'])

  # 모델 fitting
  model.fit(x_train, y_train1, epochs = 100, validation_data=(x_test, y_test1), batch_size=125)

Epoch 1/100
70/70 [==============================] - 76s 653ms/step - loss: 1.2105 - accuracy: 0.4085 - val_loss: 1.0575 - val_accuracy: 0.4335
Epoch 2/100
70/70 [==============================] - 5s 65ms/step - loss: 1.0263 - accuracy: 0.4611 - val_loss: 1.0415 - val_accuracy: 0.4252
Epoch 3/100
70/70 [==============================] - 5s 64ms/step - loss: 0.9943 - accuracy: 0.4918 - val_loss: 1.0612 - val_accuracy: 0.4480
Epoch 4/100
70/70 [==============================] - 5s 65ms/step - loss: 0.9724 - accuracy: 0.5126 - val_loss: 1.0480 - val_accuracy: 0.4439
Epoch 5/100
70/70 [==============================] - 5s 64ms/step - loss: 0.9468 - accuracy: 0.5310 - val_loss: 1.0275 - val_accuracy: 0.4688
Epoch 6/100
70/70 [==============================] - 4s 64ms/step - loss: 0.9216 - accuracy: 0.5521 - val_loss: 1.0697 - val_accuracy: 0.4574
Epoch 7/100
70/70 [==============================] - 4s 64ms/step - loss: 0.8844 - accuracy: 0.5723 - val_loss: 1.0206 - val_accuracy: 0.4657
Epoc

#Fine tune

In [7]:
## 일부 재학습 모델
def create_model_2():
  base_model = efficientnet_v2.EfficientNetV2M(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')

# base_model.trainable = False

# for layer in base_model.layers[-10:]: 
#     layer.trainable = True				

  base_model.trainable = True

  for layer in base_model.layers[:-10]: 
      layer.trainable = False				


  inputs = tf.keras.Input(shape=(224, 224, 3))

  x = base_model(inputs, training=False) # batchnorm 부분 update 방지

  x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  #x= tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  return tf.keras.Model(inputs, outputs)
  


In [8]:
with strategy.scope():
  V2M_128 = create_model_2()
  V2M_128.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  V2M_128.fit(x_train, y_train, epochs = 100, validation_data=(x_test, y_test), 
                    batch_size=64)

214201816/214201816 [==============================] - 1s 0us/step
Epoch 1/100
135/135 [==============================] - 94s 346ms/step - loss: 1.1447 - accuracy: 0.3765 - val_loss: 1.0850 - val_accuracy: 0.3940
Epoch 2/100
135/135 [==============================] - 8s 62ms/step - loss: 1.0566 - accuracy: 0.4167 - val_loss: 1.0691 - val_accuracy: 0.4158
Epoch 3/100
135/135 [==============================] - 8s 62ms/step - loss: 1.0245 - accuracy: 0.4527 - val_loss: 1.0650 - val_accuracy: 0.4116
Epoch 4/100
135/135 [==============================] - 8s 63ms/step - loss: 0.9856 - accuracy: 0.4820 - val_loss: 1.0703 - val_accuracy: 0.4189
Epoch 5/100
135/135 [==============================] - 9s 63ms/step - loss: 0.9316 - accuracy: 0.5390 - val_loss: 1.0703 - val_accuracy: 0.4158
Epoch 6/100
135/135 [==============================] - 9s 63ms/step - loss: 0.8822 - accuracy: 0.5669 - val_loss: 1.0648 - val_accuracy: 0.4553
Epoch 7/100
135/135 [==============================] - 9s 63ms/step

KeyboardInterrupt: ignored

#Tensor Board


In [41]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [42]:
import tensorflow as tf
import datetime
tf.compat.v1.disable_eager_execution()

In [43]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8635, 224, 224, 3), (8635,), (962, 224, 224, 3), (962,))

In [44]:
effic_128 = create_model_2()

effic_128.summary()

Instructions for updating:
Colocations handled automatically by placer.


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 flatten_5 (Flatten)         (None, 62720)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               8028288   
                                                                 
 dense_11 (Dense)            (None, 3)                 387       
                                                                 
Total params: 125,775,523
Trainable params: 12,542,115
Non-trainable params: 113,233,408
____________________________________

In [50]:
effic_128.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
effic_128_run_eagerly=True,

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


In [47]:
model_name = 'version.1'
current_time = datetime.datetime.now().strftime("%Y%m%d") # Y : 연도 / m : 월 / d : 일 / H : 시 / M : 분 / S : 초
current_time

'20221201'

In [51]:
log_dir = "logs/fit/" + current_time + model_name
board = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1) # epoch마다 히스토그램 계산
 
effic_128.fit(x_train, y_train, epochs=5, callbacks=[board])

Train on 8635 samples
Epoch 1/5


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


8635/8635 [==============================] - 562s 65ms/sample - loss: 1.1142 - accuracy: 0.3968
Epoch 2/5
1536/8635 [====>.........................] - ETA: 7:40 - loss: 1.0164 - accuracy: 0.4844

KeyboardInterrupt: ignored

In [ ]:
%tensorboard --logdir logs/fit

#Summary


In [59]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))


AttributeError: ignored

In [53]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [52]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [54]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [55]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [60]:
model = create_model_2() # reset our model
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
model_run_eagerly=True
EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


RuntimeError: ignored

#앙상블


In [ ]:
efficM = efficientnet_v2.EfficientNetV2M(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')

In [ ]:
efficL = efficientnet_v2.EfficientNetV2L(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')

In [ ]:
def create_model():
  base_model = efficientnet_v2.EfficientNetV2S(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')
  base_model.trainable = True
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = base_model(inputs, training=False)
  x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  #x= tf.keras.layers.Dropout(0.3)(x)
  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  return tf.keras.Model(inputs, outputs)

with strategy.scope():
  effics = create_model()
  effics.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), 
              loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
  effics.trainable
  effics.fit(x_train, y_train, epochs = 80, validation_data=(x_test, y_test), 
                   batch_size= 64)

In [ ]:
pred_s1 = effics.predict(x_test)

In [ ]:
efficL.output_shape